In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import kagglehub

# https://www.kaggle.com/datasets/mapecode/madrid-province-rent-data

# TODO: Mejoras para este ejercicio:
# - Asegurar que los datos sean consistentes: si hay valores erróneos o nulos, establecer un random_state para garantizar reproducibilidad
# - Implementar pipelines para estructurar mejor el flujo de preprocesamiento y modelado, evitando la aplicación manual de cada paso
# - Optimizar el modelo ajustando hiperparámetros con técnicas como GridSearchCV o RandomizedSearchCV
# - Utilizar Regex para validaciones: códigos postales, teléfonos, emails, etc.
# - Crear variables derivadas como precio por metro cuadrado (`precio_m2 = precio / superficie`)
# - Geolocalización: obtener coordenadas con OpenStreetMap a partir de direcciones o códigos postales y utilizarlas para análisis espaciales
# - Visualizar las viviendas en un mapa interactivo con Folium o Plotly Express para identificar patrones geográficos en los precios
# - Clusterización de zonas con K-Means o DBSCAN para detectar patrones de precios por ubicación y segmentar mejor los inmuebles
# - Evitar data leakage: Dividir los datos en train/test antes de hacer encoding, eliminar outliers o escalar,
#   asegurando que las transformaciones se ajusten sólo con el conjunto de entrenamiento y luego se apliquen en test
# - Subir el proyecto final a Kaggle

In [30]:
path = kagglehub.dataset_download("mapecode/madrid-province-rent-data")

In [31]:
csv_file = os.path.join(path, [f for f in os.listdir(path) if f.endswith(".csv")][0])
df = pd.read_csv(csv_file)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9229 entries, 0 to 9228
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   web_id             9229 non-null   int64  
 1   url                9229 non-null   object 
 2   title              9229 non-null   object 
 3   type               9229 non-null   object 
 4   price              9229 non-null   int64  
 5   deposit            5407 non-null   float64
 6   private_owner      9229 non-null   bool   
 7   professional_name  7622 non-null   object 
 8   floor_built        9229 non-null   int64  
 9   floor_area         3938 non-null   float64
 10  floor              8908 non-null   object 
 11  year_built         2893 non-null   float64
 12  orientation        4411 non-null   object 
 13  bedrooms           9229 non-null   int64  
 14  bathrooms          9229 non-null   int64  
 15  second_hand        9229 non-null   bool   
 16  lift               9229 

In [33]:
df.head(5)

,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,terrace,balcony,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update
0,99439319,https://www.idealista.com/en/inmueble/99439319/,"Flat / apartment for rent in pablo luna, 4",Flat,1400,NaN,False,Silcasas Ochocientas,60,NaN,...,True,False,False,False,False,"pablo luna, 4, Subdistrict Castilla, District ...",Chamartín,Castilla,28046.0,7 November
1,99439586,https://www.idealista.com/en/inmueble/99439586/,Penthouse for rent in calle de Bolivia,Penthouse,1300,1.0,False,Cruzity,77,NaN,...,False,False,False,False,False,"Calle de Bolivia, Subdistrict Bernabéu-Hispano...",Chamartín,Bernabéu-Hispanoamérica,28016.0,7 November
2,99439169,https://www.idealista.com/en/inmueble/99439169/,Duplex for rent in calle de la constancia,Duplex,950,1.0,False,Mm Home,72,68.0,...,True,False,False,False,False,"Calle de la constancia, Urb. no, Subdistrict P...",Chamartín,Prosperidad,28002.0,7 November
3,26925909,https://www.idealista.com/en/inmueble/26925909/,"Flat / apartment for rent in Urb. el viso, El ...",Flat,2975,1.0,False,B&H Partners,160,NaN,...,True,False,False,True,False,"Urb. el viso, Subdistrict El Viso, District Ch...",Chamartín,El Viso,NaN,7 November
4,99440018,https://www.idealista.com/en/inmueble/99440018/,Studio flat for rent in luis cabrera,Studio,650,1.0,False,Madrid en Propiedad,30,NaN,...,False,True,False,False,False,"luis cabrera, Subdistrict Prosperidad, Distric...",Chamartín,Prosperidad,28002.0,7 November


In [34]:
df.describe(include='all')

,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,terrace,balcony,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update
count,9.229000e+03,9229,9229,9229,9229.000000,5407.000000,9229,7622,9229.000000,3938.000000,...,9229,9229,9229,9229,9229,9229,9096,8458,6834.000000,9229
unique,NaN,9229,5826,10,NaN,NaN,2,1580,NaN,NaN,...,2,2,2,2,1,5576,120,165,NaN,236
top,NaN,https://www.idealista.com/en/inmueble/99439319/,Flat / apartment for rent in Castellana,Flat,NaN,NaN,False,Home Club,NaN,NaN,...,False,False,False,False,False,", Subdistrict Castellana, District Barrio de S...",Centro,Lavapiés-Embajadores,NaN,7 November
freq,NaN,1,53,7361,NaN,NaN,7622,405,NaN,NaN,...,5911,7579,7097,6882,9229,58,1307,303,NaN,1019
mean,9.133077e+07,NaN,NaN,NaN,1937.995883,1.470686,NaN,NaN,110.285405,94.357288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28076.947761,NaN
std,2.016654e+07,NaN,NaN,NaN,1615.063308,0.622239,NaN,NaN,87.183901,72.822703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171.686236,NaN
min,3.902730e+05,NaN,NaN,NaN,400.000000,1.000000,NaN,NaN,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28001.000000,NaN
25%,9.578928e+07,NaN,NaN,NaN,1000.000000,1.000000,NaN,NaN,60.000000,54.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28009.000000,NaN
50%,9.891848e+07,NaN,NaN,NaN,1400.000000,1.000000,NaN,NaN,85.000000,75.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28023.000000,NaN
75%,9.929904e+07,NaN,NaN,NaN,2300.000000,2.000000,NaN,NaN,127.000000,110.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28043.000000,NaN
